In [ ]:
import os
import sys

import pandas as pd
from datetime import datetime as dt
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time

import collections
import string
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk import pos_tag

import scipy.stats as ss
import matplotlib.pyplot as plt
import math

import re
from nltk.corpus import stopwords

from sklearn.manifold import MDS

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
def scrollWindow(driver, n = 3):
    SCROLL_PAUSE_TIME = 6

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    i = 0
    while i<n:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        i+=1

In [ ]:
def filterUrl(url):
    url = url.replace('https://gramhir.com/hosted-by-instagram/url=', '')
    url = url.replace('%3A%7C%7C%7C%7C','://').replace('%7C%7C', '/').replace('%3F', '?').replace('%3D','=').replace('%26', '&')
    return url

In [ ]:
def wrangle(text):
    text = text.encode("utf-8").decode('utf-8','ignore')
    return text

In [ ]:
path_to_driver = os.getcwd() + "\\chromedriver"
sys.path.insert(0,path_to_driver)

In [ ]:
def checkVideo(el):
    return el.find_element_by_xpath("//div[@class = 'video-icon']").is_displayed()

In [ ]:
def scrapeData(url, scroll = 3):
    wd = webdriver.Chrome('chromedriver',options=chrome_options)
    wd.get(url)
    
    time.sleep(5)
    scrollWindow(driver=wd, n=scroll)
    
    post_ul = wd.find_element_by_xpath(".//div[@class='content box-photos-wrapper']//ul")
    photoURL_list = post_ul.find_elements_by_xpath("//div[@class = 'photo']//img")
    likesComment_list = post_ul.find_elements_by_xpath("//div[@class = 'post-footer']")

    insta_list = []
    for p, lc in zip(photoURL_list, likesComment_list):
        
        insta_list.append(
            (filterUrl(p.get_attribute('src')), wrangle(p.get_attribute('alt')), *lc.text.split('\n')))
    
    data = pd.DataFrame(insta_list, columns = ['img_url', 'caption', 'n_likes_1000', 'n_comments', 'age'])
    
    return data

In [ ]:
adidas_data = scrapeData(url = 'https://gramhir.com/profile/adidas/20269764', scroll = 50)
adidas_data.to_csv('adidas_data.csv')

In [ ]:
nike_url = "https://gramhir.com/profile/nike/13460080"
nike_data = scrapeData(url = nike_url, scroll = 50)
nike_data.to_csv('nike_data.csv')